# Sample Projects
This notebook is intended as a playground to try out various python widgets, ideas etc. This notebook is publically saved on github.

In [1]:
# tax rate is fixed
TAX_RATE = .0605

In [2]:
from math import floor

def round(x):
    return floor((x * 100) + .5)/100.0

class LineItem:
    def __init__(self,label,each_amount,qty):
        self.label = label
        self.each_amount = each_amount
        self.qty = qty
        self.line_amount = round(self.each_amount * self.qty)
    def __str__(self):
        if self.label == "LTAX":
            return "{} {} x {:0,.4f} = ${:0,.2f}".format(self.label,self.qty,self.each_amount,self.line_amount)
        return "{} {} x ${:0,.2f} = ${:0,.2f}".format(self.label,self.qty,self.each_amount,self.line_amount)

def invoice_str(invoice):
    return [str(i) for i in invoice]

In [3]:
def simple_prediction(stay_nights,gross_paid,tax_paid,cln_fee,pet_fee,npets,extra_fee,rate):
    print("Trying prediction[simple,pet={}]".format(npets))
    predicted_nightly_rate = (gross_paid - tax_paid - cln_fee - extra_fee - (pet_fee * npets)) / stay_nights
    rent = (predicted_nightly_rate * stay_nights)
    pets = pet_fee * npets
    subtotal = rent + cln_fee + pets + extra_fee
    tax_calc = round(subtotal * rate)
    total = subtotal + tax_calc
    invoice = [
        LineItem("RNT",predicted_nightly_rate,stay_nights),
        LineItem("CLN",cln_fee,1),
        LineItem("PET",pet_fee,npets),
        LineItem("XFEE",extra_fee,1),
        LineItem("LTAX",rate,subtotal)]
    print("Trying invoice={}".format(invoice_str(invoice)))
    
    if total != gross_paid:
        print("Invoice FAILED!!")
        raise RuntimeError("Could not predict invoice!")
        
    print("Invoice succeeded.")
    return invoice

In [4]:
def predict_invoice(stay_nights,gross_paid,tax_paid,cln_fee=135.00,pet_fee=95.00,extra_fee=0.0, rate=.05):
    """
    Predicts the invoice line items given stay nights, gross amount paid,
    taxes paid, cleaning fee, and pet fees.  If the invoice can not be predicted,
    raise an exception.
    
    Parameters
    ----------
      stay_nights : integer
          The number of nights stayed. Used to multiple the nightly rate to determine overal rent.
      gross_paid : double
          The amount (USD) paid for the stay
      tax_paid : double
          The amount (USD) paid in taxes.  This should be total amount (rent + fees) * TAX_RATE.
      cln_fee : double
          The amount (USD) charged/paid for cleaning fees.
      pet_fee : double
          The amount (USD) charged/paid for pet fees.
    
    Returns
    -------
       A list of invoice items or raises an exception if the invoice could not be created
    """
    
#    # initialize the list of predictions
#    # a prediction is just a function that tries to return a list of invoice items
#    predictions = [simple_prediction]

    print("Predicting invoice:")
    print("   (stay_nights :: gross_paid :: tax_paid) = ({} :: ${:0,.2f} :: ${:0,.2f})".format(stay_nights,gross_paid,tax_paid))
#    prediction = predictions.pop()
    invoice = []
    npets = 0
    while True:
        try:
#            invoice = prediction(stay_nights,gross_paid,tax_paid,cln_fee,pet_fee,npets)
            invoice = simple_prediction(stay_nights,gross_paid,tax_paid,cln_fee,pet_fee,npets,extra_fee,rate)
            break
        except RuntimeError:
#            try:
#                prediction = predictions.pop()
#            except IndexError:
#                raise RuntimeError("Could not predict invoice!")
            npets += 1
            if npets==3 and rate==.05:
                raise RuntimeError("Could not predict invoice!")
            elif npets==3:
                return predict_invoice(stay_nights,gross_paid,tax_paid,cln_fee,pet_fee,extra_fee,rate=.0605)

    return invoice

In [5]:
import csv

with open('payments.tsv') as payments_file:
    values_tsv = csv.reader(payments_file, delimiter='\t')
    data=[row for row in values_tsv]
    data=data[1:]
data

[['HA-7G4ZNN', '20201213', '8', '1518.3', '48.58', '1469.72', '72.3', '0'],
 ['HA-DJVVLZ', '20201214', '4', '645.75', '20.66', '625.09', '30.75', '0'],
 ['HA-HR9X86',
  '20201226',
  '7',
  '1748.67',
  '55.95',
  '1692.72',
  '83.27',
  '-29.6'],
 ['HA-RHS034',
  '20201228',
  '9',
  '2340.45',
  '74.51',
  '2265.94',
  '111.45',
  '-10'],
 ['HA-H7F8FJ', '20210102', '7', '1756.23', '56.19', '1700.04', '83.63', '0'],
 ['HA-9NZ0ZG', '20210103', '4', '1222.2', '39.1', '1183.1', '58.2', '0']]

In [6]:
def invoice_label(row):
    return "{} co {}:{}".format(row[0],row[1],row[3])

def to_invoice(row):
    try:
        return invoice_str(predict_invoice(stay_nights=int(row[2]),
                                          gross_paid=float(row[3]),
                                          tax_paid=float(row[6]),
                                          extra_fee=float(row[7])))
    except:
        return "INVOICE FAIL"

results = {invoice_label(row):to_invoice(row) for row in data}
results

Predicting invoice:
   (stay_nights :: gross_paid :: tax_paid) = (8 :: $1,518.30 :: $72.30)
Trying prediction[simple,pet=0]
Trying invoice=['RNT 8 x $163.88 = $1,311.00', 'CLN 1 x $135.00 = $135.00', 'PET 0 x $95.00 = $0.00', 'XFEE 1 x $0.00 = $0.00', 'LTAX 1446.0 x 0.0500 = $72.30']
Invoice succeeded.
Predicting invoice:
   (stay_nights :: gross_paid :: tax_paid) = (4 :: $645.75 :: $30.75)
Trying prediction[simple,pet=0]
Trying invoice=['RNT 4 x $120.00 = $480.00', 'CLN 1 x $135.00 = $135.00', 'PET 0 x $95.00 = $0.00', 'XFEE 1 x $0.00 = $0.00', 'LTAX 615.0 x 0.0500 = $30.75']
Invoice succeeded.
Predicting invoice:
   (stay_nights :: gross_paid :: tax_paid) = (7 :: $1,748.67 :: $83.27)
Trying prediction[simple,pet=0]
Trying invoice=['RNT 7 x $222.86 = $1,560.00', 'CLN 1 x $135.00 = $135.00', 'PET 0 x $95.00 = $0.00', 'XFEE 1 x $-29.60 = $-29.60', 'LTAX 1665.4 x 0.0500 = $83.27']
Invoice succeeded.
Predicting invoice:
   (stay_nights :: gross_paid :: tax_paid) = (9 :: $2,340.45 :: $111.

{'HA-7G4ZNN co 20201213:1518.3': ['RNT 8 x $163.88 = $1,311.00',
  'CLN 1 x $135.00 = $135.00',
  'PET 0 x $95.00 = $0.00',
  'XFEE 1 x $0.00 = $0.00',
  'LTAX 1446.0 x 0.0500 = $72.30'],
 'HA-DJVVLZ co 20201214:645.75': ['RNT 4 x $120.00 = $480.00',
  'CLN 1 x $135.00 = $135.00',
  'PET 0 x $95.00 = $0.00',
  'XFEE 1 x $0.00 = $0.00',
  'LTAX 615.0 x 0.0500 = $30.75'],
 'HA-HR9X86 co 20201226:1748.67': ['RNT 7 x $222.86 = $1,560.00',
  'CLN 1 x $135.00 = $135.00',
  'PET 0 x $95.00 = $0.00',
  'XFEE 1 x $-29.60 = $-29.60',
  'LTAX 1665.4 x 0.0500 = $83.27'],
 'HA-RHS034 co 20201228:2340.45': ['RNT 9 x $233.78 = $2,104.00',
  'CLN 1 x $135.00 = $135.00',
  'PET 0 x $95.00 = $0.00',
  'XFEE 1 x $-10.00 = $-10.00',
  'LTAX 2229.0 x 0.0500 = $111.45'],
 'HA-H7F8FJ co 20210102:1756.23': ['RNT 7 x $219.66 = $1,537.60',
  'CLN 1 x $135.00 = $135.00',
  'PET 0 x $95.00 = $0.00',
  'XFEE 1 x $0.00 = $0.00',
  'LTAX 1672.6 x 0.0500 = $83.63'],
 'HA-9NZ0ZG co 20210103:1222.2': ['RNT 4 x $257.25 